Нейроконсультант по Правилам безопасности опасных производственных объектов, на которых используются подъемные сооружения

In [ ]:
!pip install -q langchain_openai==0.0.2 faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain_community==0.0.11 langchain==0.1.0 cohere

In [2]:
import os
from google.colab import userdata
from openai import OpenAI

key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key
client = OpenAI()

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gdown
import re
from langchain.docstore.document import Document

In [ ]:
def get_direct_link(sharing_link):
    id = re.findall(r'/d/([a-zA-Z0-9_-]{25,})', sharing_link)  # Находим идентификатор файла в ссылке
    if id:
        return f'https://drive.google.com/uc?export=download&id={id[0]}'  # Формируем прямую ссылку
    else:
        return None

# Ссылка на файл, сохраненный в TXT
sharing_link = 'https://drive.google.com/file/d/1MUEAhTC33dVd0d2Dig9q07dJDL9nmYCG/view?usp=sharing' # изменена
direct_link = get_direct_link(sharing_link)
print(direct_link)
gdown.download(direct_link, None, quiet = True)

with open('safety.txt', 'r', encoding='utf-8') as f:
    doc = f.read()

In [33]:
# RecursiveCharacterTextSplitter
chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
for chunk in splitter.split_text(doc):
    chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
print(len(chunks))

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

255


In [34]:
query = "Как проводятся статические испытания крана стрелового типа?"
results = db.similarity_search(query, k=4)

print("**отрезок №1** :", results[0].page_content[:150])
print("\n**отрезок №2** :", results[1].page_content[:150])
print("\n**отрезок №3** :", results[2].page_content[:150])
print("\n**отрезок №4** :", results[3].page_content[:150])

**отрезок №1** : грузов, могут проводиться при помощи специальных приспособлений, позволяющих создать испытательную нагрузку без применения груза. Порядок проведения т

**отрезок №2** : этими же вылетами при условии удовлетворительных результатов технического освидетельствования разрешается последующая работа крана. 175. Для проведени

**отрезок №3** : крана; при этом у крана с консолями каждая консоль испытывается отдельно. При наличии остаточной деформации, явившейся следствием испытания крана груз

**отрезок №4** : от первоначальных значений проверяется по оголовку стрелы. Если в течение 10 минут поднятый груз не опустится на землю, а также не будет обнаружено тр


In [38]:
def answer_index(system, topic, search_index, verbose=0):
    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + \
                                                        doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'The answer to the question in the language in which it was asked. '
                                    f"Don't mention the document with the information for the answer. "
                                    f'Document with information for the response: {message_content}\n\n'
                                    f'Question: \n{topic}'}
    ]
    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=0
    )

    print(f'\nВопрос: {completion.usage.prompt_tokens} токенов. '
          f'\nОтвет: {completion.usage.completion_tokens} токенов. '
          f'\nВсего: {completion.usage.total_tokens} токенов.\n')
    return completion.choices[0].message.content


system = 'You are a consultant on the safety rules of hazardous production facilities where lifting facilities are used'

In [39]:
topic= "Как проводятся статические испытания крана стрелового типа?"
answer_index(system, topic, db)


Вопрос: 1795 токенов. 
Ответ: 122 токенов. 
Всего: 1917 токенов.



'Статические испытания крана стрелового типа проводятся в положении, соответствующем наибольшей грузоподъемности крана, при установленном стреловом оборудовании. Груз поднимается на высоту 50-100 мм, и замеры остаточных деформаций осуществляются в соответствии с установленными требованиями.'

In [40]:
topic= "Как осуществляются динамические испытания ПС?"
answer_index(system, topic, db)


Вопрос: 1853 токенов. 
Ответ: 176 токенов. 
Всего: 2029 токенов.



'Динамические испытания подъемных сооружений (ПС) проводятся с грузом, масса которого на 10 процентов превышает его паспортную грузоподъемность, с целью проверки действия его механизмов и тормозов. При динамических испытаниях ПС производятся многократные подъем и опускание груза, а также проверка действия всех других механизмов при совмещении рабочих движений, предусмотренных руководством по эксплуатации ПС.'

In [41]:
topic= "Какие требования к проверке состояния пакетирующих стропов?"
answer_index(system, topic, db)


Вопрос: 1868 токенов. 
Ответ: 178 токенов. 
Всего: 2046 токенов.



'Состояние пакетирующих стропов должно проверяться перед каждой операцией подъема запакетированного груза путем подъема пакета в соответствии с утвержденными схемами строповки на высоту 100-200 мм от поверхности, на которой расположен пакет, и выдержки в таком положении не менее 30 секунд. Если форма и целостность пакета груза в течение времени выдержки не изменились, то строп признается годным к дальнейшему использованию.'